# Optimizing Python Code 


## Sparse Matrices

### Why regular matrices  suck sometimes (Especially for co-occurences)

Our co-occurence matrices are sparse because they have a TON of zeroes in them. 

It's a waste of memory to store those zeroes!

If we have 42k different tokens in our vocab, our cooccurence matrix is 42k by 42k. 

42,000 x 42,000 = 1,764,000,000

If an int takes 4 bytes in python, we need (1,764,000,000 * 4 = 7,056,000,000 Bytes of memory in RAM TO STORE THE MATRIX)

#### That's 7 GIGABYTES OF MEMORY to store the matrix for hw3, most of which is just storing 0 values! 

Well, since we know that those values are 0... Can we store these matrices in a smarter way? YES WE CAN

### Solution: Sparse Matrices (Store only nonzero elements of a matrix)

If we instead only store the locations and values of any spots in a matrix that aren't zero, we can vastly cut down how much storage we need. With a CSR matrix, you can simplify the amount of numbers you store to:

Num_of_nonzero_vals_in_matrix * 3 {3 comes from the number of lists required to store a sparse matrix}

So say there were as many as 500k nonzero elements in your co-occurance matrix, 500k*3 *4bytes= 6mil bytes= 6 MB

#### 6MB is <<<<<<<<<<<<< 7 GB, which will really help your program run way faster!

Also because many math operations we do on matrices leave 0's as 0's, sparse matrices make things like SVD's much quicker because we process less values. For instance, creating a PPMI with a sparse matrix let's you skip over calculating all the 0's, which would've been 0 anyway!

Now there are many forms of sparse matrices (7 in scipy), each with their own pros and cons. Usually it's more complicated to do operations on them, and if a matrix didn't have many 0's, it would take 3 or more times the memory than the regular dense representation. 

Take a look at the quick tutorial linked below to get set up with how CSR matrices work (great choice for hw3), and the longer tutorial if you are more curious. HOWEVER, the quick tutorial doesn't go over how to build a sparse matrix from scratch (skipping making that dense matrix). It's easier than you think, but if you need extra help come to OH!

Quick Tutorial: https://machinelearningmastery.com/sparse-matrices-for-machine-learning/

Longer Tutorial: https://medium.com/swlh/an-in-depth-introduction-to-sparse-matrix-a5972d7e8c86



## Optimizing Python Code

## Installations

Line profiler is a SUPER useful tool that will tell you how long each line in your python function is taking (to spot pesky bottlenecks)! 

Run it as:

%lprun -f FUNCTION_NAME FUNCTION_NAME(INPUTS)

in a jupyter code cell

In [ ]:
! pip install line-profiler
%load_ext line_profiler

In [2]:
import numpy as np
import scipy as sp
import numba 

## Basic Tips

### 1. List Comprehensions

src: https://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/

In [9]:
input_list = [1, 2, -3]
output_list = []
for x in input_list:
    if x >= 0:
        output_list.append(1)
    else:
        output_list.append(0)
output_list

[1, 1, 0]

In [10]:
output_list = [1 if x >= 0 else 0 for x in input_list]
output_list

[1, 1, 0]

### 2. Work with Python built-in functions. They're already optimized! (Built in Cython)

What is Cython? People rewrote Python functions in C so that they run wayyyy faster: https://cython.org

In [12]:
# Slow manual for loop
sentence_list = ['This ', 'is ', 'a ', 'sentence.']
sentence = ''
for i in sentence_list:
    sentence += i
sentence

'This is a sentence.'

In [38]:
# Much quicker function written in Cython by the devs
sentence = ''.join(sentence_list)
sentence

'This is a sentence.'

## 3. Remember your data structures!

In [24]:
## Need to lookup things quickly? Don't forget your dicts!

In [39]:
corpus = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."
corpus

"Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."

In [40]:
# Nice way to get indexable vocab list

split_corp = corpus.split() # Split corpus into words
corpus_size = len(split_corp) # Get size of corpus (for some operations)
vocab = set(split_corp) # Remove duplicate words
vocab = list(vocab) # Turn into list so you can have a dict
vocab_idx = {token: index for index, token in enumerate(vocab)} # Dict so you can quickly build a co-occurence mat
vocab_idx

{'software': 0,
 'but': 1,
 'since': 2,
 'a': 3,
 'it': 4,
 'text': 5,
 'Ipsum.': 6,
 'specimen': 7,
 'industry.': 8,
 "industry's": 9,
 'survived': 10,
 'release': 11,
 'in': 12,
 'not': 13,
 'recently': 14,
 'been': 15,
 'unchanged.': 16,
 'also': 17,
 'type': 18,
 'Lorem': 19,
 'leap': 20,
 'dummy': 21,
 'essentially': 22,
 'standard': 23,
 'the': 24,
 'passages,': 25,
 'printing': 26,
 'an': 27,
 'printer': 28,
 'desktop': 29,
 'took': 30,
 'like': 31,
 'with': 32,
 'versions': 33,
 'five': 34,
 '1960s': 35,
 'and': 36,
 'galley': 37,
 'Letraset': 38,
 'remaining': 39,
 'is': 40,
 'PageMaker': 41,
 'centuries,': 42,
 'of': 43,
 'typesetting': 44,
 'publishing': 45,
 'including': 46,
 'electronic': 47,
 'to': 48,
 'was': 49,
 'has': 50,
 'more': 51,
 'containing': 52,
 'ever': 53,
 '1500s,': 54,
 'into': 55,
 'unknown': 56,
 'Ipsum': 57,
 'book.': 58,
 'popularised': 59,
 'make': 60,
 'when': 61,
 'typesetting,': 62,
 'It': 63,
 'simply': 64,
 'Aldus': 65,
 'only': 66,
 'scrambled':

## 4. Numpy Matrices >>>>> Lists

In [29]:
np.random.seed(445)

In [34]:
x = np.random.choice([False, True], size=100000000)
x

array([False, False, False, ...,  True,  True, False])

In [35]:
# Slow by hand function
def count_transitions(x) -> int:
    count = 0
    for i, j in zip(x[:-1], x[1:]):
        if j and not i:
            count += 1
    return count
count_transitions(x)

25000897

In [41]:
# Fast numpy array version :) FIFTY-FIVE TIMES FASTER
np.count_nonzero(x[:-1] < x[1:])

25000897

In [23]:
from timeit import timeit
setup = 'from __main__ import count_transitions, x; import numpy as np'
num = 1000
t1 = timeit('count_transitions(x)', setup=setup, number=num)
t2 = timeit('np.count_nonzero(x[:-1] < x[1:])', setup=setup, number=num)
print('Speed difference: {:0.1f}x'.format(t1 / t2))

Speed difference: 55.3x


## 5. non-vectorized vs vectorized code

Vectorization guide: https://realpython.com/numpy-array-programming/



This will take an array representing M points in N dimensions, and return the M x M matrix of pairwise distances. This is a nice test function for a few reasons. First of all, it's a very clean and well-defined test. Second of all, it illustrates the kind of array-based operation that is common in statistics, datamining, and machine learning. Third, it is a function that results in large memory consumption if the standard numpy broadcasting approach is used (it requires a temporary array containing M * M * N elements), making it a good candidate for an alternate approach.

In [7]:
X = np.random.random((1000, 3))
print(X)

[[0.50242489 0.17970434 0.27677217]
 [0.02365    0.24098798 0.6849179 ]
 [0.54919357 0.99403727 0.40053637]
 ...
 [0.44405792 0.15009429 0.83797054]
 [0.50183771 0.81814624 0.62062579]
 [0.51114686 0.73207088 0.77506034]]


### Python For Loop

In [4]:
def pairwise_python(X):
    M = X.shape[0]
    N = X.shape[1]
    D = np.empty((M, M), dtype=float)
    for i in range(M):
        for j in range(M):
            d = 0.0
            for k in range(N):
                tmp = X[i, k] - X[j, k]
                d += tmp * tmp
            D[i, j] = np.sqrt(d)
    return D
%timeit pairwise_python(X)

2.15 s ± 6.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%lprun -f pairwise_python pairwise_python(X)

In [5]:
def pairwise_numpy(X):
    return np.sqrt(((X[:, None, :] - X) ** 2).sum(-1))
%timeit pairwise_numpy(X)

24.6 ms ± 439 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
from sklearn.metrics import euclidean_distances
%timeit euclidean_distances(X, X)

3.9 ms ± 34.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
